In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery – Hierarchical model, with ADVI

In [2]:
output_folder = join('results', 'parameter-recovery_hierarchical', 'advi')
label = 'hierarchical_advi'

## Define parameter ranges from 2011 estimates

In [3]:
bounds = dict(v=[0.000015, 0.00015],
              gamma=[-1, 1],
              s=[0.004, 0.011],
              tau=[0.1, 1.25])

In [4]:
# Determine low, medium, high mean values as 10th, 50th and 90th percentile of these ranges
means = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
means

{'v': array([2.850e-05, 8.250e-05, 1.365e-04]),
 'gamma': array([-8.00000000e-01,  5.55111512e-17,  8.00000000e-01]),
 's': array([0.0047, 0.0075, 0.0103]),
 'tau': array([0.215, 0.675, 1.135])}

In [5]:
sds = dict(v=0.00005,
           s=0.001,
           gamma=0.3,
           tau=0.2)

In [6]:
parameter_info = {parameter: dict(mu=dict(low=means[parameter][0],
                                          medium=means[parameter][1],
                                          high=means[parameter][2]),
                                  sd=sds[parameter],
                                  bounds=tuple(bounds[parameter]))
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [7]:
partial_output_folder = join(output_folder, 'partial')

simulate_group_args = dict(kind='hierarchical',
                           n_individuals=20,
                           n_trials=50, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='hierarchical',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='VI',
                draws=2000,
                n_vi=50000,
                progressbar=False)

wrapper = partial(pr.recover_and_save_hierarchical,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args)

In [ ]:
from multiprocessing import Pool

n_cores = 8
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_hierarchical_parameter_sets(parameter_info))

Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...
Generating hierarchical model for 20 subjects...


WARNING (theano.gof.compilelock): Overriding existing lock by dead process '27032' (I am process '27090')
INFO (theano.gof.compilelock): Waiting for existing lock by process '27092' (I am process '27091')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27092' (I am process '27096')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27092' (I am process '27094')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27092' (I am process '27093')
INFO (theano.go

Fitting 1 model(s) using VI...
  Fitting model 1 of 1...


INFO (theano.gof.compilelock): Waiting for existing lock by process '27090' (I am process '27091')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27090' (I am process '27096')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27090' (I am process '27094')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '27090' (I am process '27093')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x

Fitting 1 model(s) using VI...
  Fitting model 1 of 1...


...

Finished [100%]: Average Loss = 8,071.4


/!\ Automatically setting parameter precision...
   gamma  gamma_hpd_2.5  gamma_hpd_97.5  gamma_mu  gamma_mu_hpd_2.5  \
0   0.26       0.086927        0.510296  0.641902          0.534961   

   gamma_mu_hpd_97.5         s  s_hpd_2.5  s_hpd_97.5      s_mu  \
0             0.7505  0.010634   0.009801    0.012151  0.010142   

       ...        tau_hpd_97.5    tau_mu  tau_mu_hpd_2.5  tau_mu_hpd_97.5  \
0      ...             0.48443  0.287038         0.21697         0.360622   

          v  v_hpd_2.5  v_hpd_97.5      v_mu  v_mu_hpd_2.5  v_mu_hpd_97.5  
0  0.000154   0.000136    0.000167  0.000128      0.000113       0.000142  

[1 rows x 28 columns]
level_set high
parameter_set {'mu': 0.0001365, 'sd': 5e-05, 'bounds': (1.5e-05, 0.00015)}
recovered 0.00012754482323183554
Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using VI...
  Fitting model 1 of 1...


Finished [100%]: Average Loss = 7,291


/!\ Automatically setting parameter precision...
   gamma  gamma_hpd_2.5  gamma_hpd_97.5  gamma_mu  gamma_mu_hpd_2.5  \
0   0.48       0.425082        0.544538   0.67394          0.566008   

   gamma_mu_hpd_97.5         s  s_hpd_2.5  s_hpd_97.5      s_mu  \
0           0.775539  0.003024   0.002797    0.003451  0.002893   

       ...        tau_hpd_97.5    tau_mu  tau_mu_hpd_2.5  tau_mu_hpd_97.5  \
0      ...            0.981068  0.512608        0.421553         0.599224   

          v  v_hpd_2.5  v_hpd_97.5      v_mu  v_mu_hpd_2.5  v_mu_hpd_97.5  
0  0.000147   0.000143    0.000152  0.000127      0.000107       0.000147  

[1 rows x 28 columns]
level_set high
parameter_set {'mu': 0.0001365, 'sd': 5e-05, 'bounds': (1.5e-05, 0.00015)}
recovered 0.00012659418448934137
Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using VI...
  Fitting model 1 of 1...


Finished [100%]: Average Loss = 8,037.1


## Combine results

In [ ]:
filenames = [file for file in listdir(partial_output_folder)
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join(partial_output_folder, file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join(output_folder, 'parameter-recovery_', label, '.csv'))
parameter_recovery.head()